In [3]:
import pandas as pd
import json

In [4]:
df = pd.read_csv("/data/raw_stock_indicator.csv")

In [5]:
def json_parser(x):
    try:
        return json.loads(x)["symbol"]
    except Exception as e:
        print(e)
        return ""

In [6]:
df["company_symbol"] = df["info_json"].map(lambda x: json_parser(x))  

In [14]:
df["company_symbol"].unique()

array(['LTIM.NS', '20MICRONS.NS', '21STCENMGM.NS', ..., 'ZODIACLOTH.NS',
       'ZOTA.NS', 'ZUARI.NS'], dtype=object)

In [15]:
df.columns

Index(['Tax Effect Of Unusual Items', 'Tax Rate For Calcs',
       'Normalized EBITDA', 'Total Unusual Items',
       'Total Unusual Items Excluding Goodwill',
       'Net Income From Continuing Operation Net Minority Interest',
       'Reconciled Depreciation', 'Reconciled Cost Of Revenue', 'EBIT',
       'Net Interest Income', 'Interest Expense', 'Normalized Income',
       'Net Income From Continuing And Discontinued Operation',
       'Total Expenses', 'Diluted Average Shares', 'Basic Average Shares',
       'Diluted EPS', 'Basic EPS', 'Diluted NI Availto Com Stockholders',
       'Net Income Common Stockholders', 'Otherunder Preferred Stock Dividend',
       'Net Income', 'Minority Interests',
       'Net Income Including Noncontrolling Interests',
       'Net Income Continuous Operations', 'Tax Provision', 'Pretax Income',
       'Other Non Operating Income Expenses', 'Special Income Charges',
       'Restructuring And Mergern Acquisition',
       'Net Non Operating Interest Inco

In [16]:
cols = ["Total Revenue", "Operating Revenue", "Gross Profit", "Year", "Month", "company_symbol" ,"company_symbol", "Open", "Volume"]

In [17]:
df1 = df[cols]
df1["year_month"] = (10*df1["Year"])+df1["Month"]
df1

<ipython-input-17-90b8b26be631>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["year_month"] = (10*df1["Year"])+df1["Month"]


,Total Revenue,Operating Revenue,Gross Profit,Year,Month,company_symbol,company_symbol,Open,Volume,year_month
0,8.702100e+10,8.702100e+10,3.103400e+10,2023,6,LTIM.NS,LTIM.NS,5084.918182,581634.0,20236
1,8.691000e+10,8.691000e+10,3.110300e+10,2023,3,LTIM.NS,LTIM.NS,4603.711157,328337.0,20233
2,8.620000e+10,8.620000e+10,3.147800e+10,2022,12,LTIM.NS,LTIM.NS,NaN,NaN,20232
3,4.836700e+10,4.836700e+10,1.751800e+10,2022,9,LTIM.NS,LTIM.NS,4345.868216,410491.0,20229
4,4.522800e+10,4.522800e+10,1.650500e+10,2022,6,LTIM.NS,LTIM.NS,4019.437045,374541.0,20226
...,...,...,...,...,...,...,...,...,...,...
7310,1.205380e+10,1.205380e+10,4.469900e+09,2023,6,ZUARI.NS,ZUARI.NS,149.000000,59962.0,20236
7311,1.338910e+10,1.338910e+10,4.684400e+09,2023,3,ZUARI.NS,ZUARI.NS,121.900002,98839.0,20233
7312,1.416170e+10,1.416170e+10,5.367000e+09,2022,12,ZUARI.NS,ZUARI.NS,NaN,NaN,20232
7313,5.016200e+09,5.016200e+09,1.144400e+09,2022,9,ZUARI.NS,ZUARI.NS,168.600006,457551.0,20229


In [21]:
df1['price_change'] = df1.groupby(['year_month', 'company_symbol'])['Open'].shift(1)

ValueError: Grouper for 'company_symbol' not 1-dimensional

In [22]:
df1

,Total Revenue,Operating Revenue,Gross Profit,Year,Month,company_symbol,company_symbol,Open,Volume,year_month,price_change
0,8.702100e+10,8.702100e+10,3.103400e+10,2023,6,LTIM.NS,LTIM.NS,5084.918182,581634.0,20236,NaN
1,8.691000e+10,8.691000e+10,3.110300e+10,2023,3,LTIM.NS,LTIM.NS,4603.711157,328337.0,20233,NaN
2,8.620000e+10,8.620000e+10,3.147800e+10,2022,12,LTIM.NS,LTIM.NS,NaN,NaN,20232,NaN
3,4.836700e+10,4.836700e+10,1.751800e+10,2022,9,LTIM.NS,LTIM.NS,4345.868216,410491.0,20229,NaN
4,4.522800e+10,4.522800e+10,1.650500e+10,2022,6,LTIM.NS,LTIM.NS,4019.437045,374541.0,20226,NaN
...,...,...,...,...,...,...,...,...,...,...,...
7310,1.205380e+10,1.205380e+10,4.469900e+09,2023,6,ZUARI.NS,ZUARI.NS,149.000000,59962.0,20236,100.599998
7311,1.338910e+10,1.338910e+10,4.684400e+09,2023,3,ZUARI.NS,ZUARI.NS,121.900002,98839.0,20233,291.600006
7312,1.416170e+10,1.416170e+10,5.367000e+09,2022,12,ZUARI.NS,ZUARI.NS,NaN,NaN,20232,NaN
7313,5.016200e+09,5.016200e+09,1.144400e+09,2022,9,ZUARI.NS,ZUARI.NS,168.600006,457551.0,20229,291.000000
